# A notebook to document and try some of the sqoop issues I've experienced in the past
begin by initializing the variables for python and for sqoop

In [1]:
### Setup parameters
pfile = '/user/kesj/config/.myp'
user='kesj'
dbip = 'jdbc:db2://10.96.37.166:60100/FDW2F'
bdir = '/home/kesj/work/sqoopIssues'
hdir = '/user/kesj/sqooptest2/'

### check connection is working

In [2]:
dbList = !sqoop list-databases --connect {dbip} --username {user} --password-file {pfile}
dbList=dbList[4:]
print "There are {0} databases (schemas) within this DB".format(len(dbList))

There are 69 databases (schemas) within this DB


In [4]:
dbList

['15/11/05 13:23:39 ERROR manager.Db2Manager: Failed to list databases',
 'com.ibm.db2.jcc.am.DisconnectNonTransientConnectionException: [jcc][t4][2057][11264][3.65.119] The application server rejected establishment of the connection.',
 'An attempt was made to access a database, FDW2F, which was either not found or does not support transactions. ERRORCODE=-4499, SQLSTATE=08004',
 '\tat com.ibm.db2.jcc.am.cd.a(cd.java:321)',
 '\tat com.ibm.db2.jcc.am.cd.a(cd.java:387)',
 '\tat com.ibm.db2.jcc.t4.z.u(z.java:1874)',
 '\tat com.ibm.db2.jcc.t4.z.n(z.java:668)',
 '\tat com.ibm.db2.jcc.t4.z.a(z.java:475)',
 '\tat com.ibm.db2.jcc.t4.z.a(z.java:118)',
 '\tat com.ibm.db2.jcc.t4.b.c(b.java:1351)',
 '\tat com.ibm.db2.jcc.t4.b.b(b.java:1222)',
 '\tat com.ibm.db2.jcc.t4.b.b(b.java:789)',
 '\tat com.ibm.db2.jcc.t4.b.a(b.java:761)',
 '\tat com.ibm.db2.jcc.t4.b.a(b.java:422)',
 '\tat com.ibm.db2.jcc.t4.b.a(b.java:397)',
 '\tat com.ibm.db2.jcc.t4.b.<init>(b.java:335)',
 '\tat com.ibm.db2.jcc.DB2SimpleD

In [3]:
[a for a in dbList if a.startswith('FDWDRVDECSCF')]

[]

### Likewise get the list of tables

In [ ]:
tblList = !sqoop list-tables --connect {dbip} --username {user} --password-file {pfile}
#jdbc:db2://10.96.37.166:60100/FDW2P 
tblList = tblList[5:]
print "There are {0} tables in these schemas.".format(len(tblList))
[a for a in tblList if a.startswith('HLDI')]

### check what is present in this directory

In [ ]:
!hadoop fs -ls -R -h {hdir}

### remove everything in this directory

In [28]:
# remove everything
!hadoop fs -rm -r {hdir}

15/01/23 12:23:09 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 1440 minutes, Emptier interval = 0 minutes.
Moved: 'hdfs://nameservice1/user/kesj/sqooptest' to trash at: hdfs://nameservice1/user/kesj/.Trash/Current


### try to pull all of FDWATOMCAE.AUTO_EST_SECT (a small file)
1. single threaded; as avro
2. single-threaded; text
3. parallel


In [ ]:
!sqoop import --connect {dbip} --username {user} --password-file {pfile} --table FDWATOMCAE.AUTO_EST_SECT -m 1 --target-dir sqooptest/avrotest/ae_sect_single -as-avrodatafile 

In [ ]:
!sqoop import --connect {dbip} --username {user} --password-file {pfile} --table FDWATOMCAE.AUTO_EST_SECT -m 1 --target-dir sqooptest/texttest/ae_sect_single_text  

In [ ]:
# in parallel
!sqoop import --connect {dbip} --username {user} --password-file {pfile} --table FDWATOMCAE.AUTO_EST_SECT -m 4 --target-dir sqooptest/texttest/ae_sect_quad --split-by 'AUTO_EST_SECT_DIM_ID'
#!sqoop import --connect {dbip} --username {user} --password-file {pfile} --table FDWATOMCAE.AUTO_EST_SECT -m 4 --target-dir sqooptest/avrotest/ae_sect_quad --split-by 'AUTO_EST_SECT_DIM_ID' --as-avrodatafile

## Now begin to show examples that fail

In [ ]:
# Y1753CAA
tdir = hdir+'/DETL/'+'y1753caa'
print "putting data into this directory:", tdir
!sqoop import --connect {dbip} --username {user} --password-file {pfile} --table FDWATOMCAE.DETL --where "LOS_EST_DIM_ID in (SELECT LOS_EST_DIM_ID FROM FDWATOMCAE.LOS_EST WHERE VNDR_VEH_CD='Y1753CAA')" --target-dir {tdir} -m 4 --split-by "DETL_DIM_ID" --fields-terminated-by "\t"

In [31]:
# Y1753AAA
tdir = hdir+'/DETL/'+'y1753aaa'
print "putting data into this directory:", tdir
!sqoop import --connect {dbip} --username {user} --password-file {pfile} --table FDWATOMCAE.DETL --where "LOS_EST_DIM_ID in (SELECT LOS_EST_DIM_ID FROM FDWATOMCAE.LOS_EST WHERE VNDR_VEH_CD='Y1753AAA')" --target-dir {tdir} -m 4 --split-by "DETL_DIM_ID" --fields-terminated-by "\t"

putting data into this directory: /user/kesj/sqooptest//DETL/y1753aaa
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
15/01/23 12:33:47 INFO sqoop.Sqoop: Running Sqoop version: 1.4.4-cdh5.1.2
15/01/23 12:33:48 INFO manager.SqlManager: Using default fetchSize of 1000
15/01/23 12:33:48 INFO tool.CodeGenTool: Beginning code generation
15/01/23 12:33:50 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM FDWATOMCAE.DETL AS t WHERE 1=0
15/01/23 12:33:50 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM FDWATOMCAE.DETL AS t WHERE 1=0
15/01/23 12:33:51 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce
Note: /tmp/sqoop-kesj/compile/aba950152c421b1a4680eeb6ee935057/FDWATOMCAE_DETL.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
15/01/23 12:33:52 INFO orm.CompilationManager: Writing jar file: /tmp/sqoop-kesj/compile/aba950152c421b1a4680eeb6ee935057/FDWA

### hunting down the line that fails

In [30]:
# Y1753AAA
tdir = hdir+'/DETL/'+'y1753aaa_hunting'+'works'
print "putting data into this directory:", tdir
!sqoop import --connect {dbip} --username {user} --password-file {pfile} --table FDWATOMCAE.DETL --where "LOS_EST_DIM_ID in (SELECT LOS_EST_DIM_ID FROM FDWATOMCAE.LOS_EST WHERE VNDR_VEH_CD='Y1753AAA') and DETL_DIM_ID <= 1533425450" --target-dir {tdir} -m 1 --fields-terminated-by "\t"

putting data into this directory: /user/kesj/sqooptest//DETL/y1753aaa_huntingworks
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
15/01/23 12:25:34 INFO sqoop.Sqoop: Running Sqoop version: 1.4.4-cdh5.1.2
15/01/23 12:25:35 INFO manager.SqlManager: Using default fetchSize of 1000
15/01/23 12:25:35 INFO tool.CodeGenTool: Beginning code generation
15/01/23 12:25:36 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM FDWATOMCAE.DETL AS t WHERE 1=0
15/01/23 12:25:36 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM FDWATOMCAE.DETL AS t WHERE 1=0
15/01/23 12:25:36 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce
Note: /tmp/sqoop-kesj/compile/e15576189541ea69850a9070086ba8c2/FDWATOMCAE_DETL.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
15/01/23 12:25:38 INFO orm.CompilationManager: Writing jar file: /tmp/sqoop-kesj/compile/e15576189541ea69850a9070

In [29]:
# Y1753AAA
tdir = hdir+'/DETL/'+'y1753aaa_hunting'+'fails'
print "putting data into this directory:", tdir
!sqoop import --connect {dbip} --username {user} --password-file {pfile} --table FDWATOMCAE.DETL --where "LOS_EST_DIM_ID in (SELECT LOS_EST_DIM_ID FROM FDWATOMCAE.LOS_EST WHERE VNDR_VEH_CD='Y1753AAA') and DETL_DIM_ID = 1533425450" --target-dir {tdir} -m 1 --fields-terminated-by "\t"

putting data into this directory: /user/kesj/sqooptest//DETL/y1753aaa_huntingfails
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
15/01/23 12:23:28 INFO sqoop.Sqoop: Running Sqoop version: 1.4.4-cdh5.1.2
15/01/23 12:23:29 INFO manager.SqlManager: Using default fetchSize of 1000
15/01/23 12:23:29 INFO tool.CodeGenTool: Beginning code generation
15/01/23 12:23:31 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM FDWATOMCAE.DETL AS t WHERE 1=0
15/01/23 12:23:31 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM FDWATOMCAE.DETL AS t WHERE 1=0
15/01/23 12:23:31 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce
Note: /tmp/sqoop-kesj/compile/450105bff4bb0d12b8032ffea4ba6af0/FDWATOMCAE_DETL.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
15/01/23 12:23:32 INFO orm.CompilationManager: Writing jar file: /tmp/sqoop-kesj/compile/450105bff4bb0d12b8032ffe